In [2]:
import pandas as pd
import numpy as np
import re
import gensim
import torch
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances

from collections import Counter

In [3]:
metadata = pd.read_csv('./res/metadata.csv')

In [4]:
#metadata = metadata.drop(['magazine_id', 'user_id', 'title', 'display_url','sub_title', 'reg_ts', 'article_id', 'id', 'date', 'hour', 'weekday', 'view', 'recent_view'], axis = 1)

In [5]:
# Nan값 제거
metadata[['article_tag_list']] = metadata[['article_tag_list']].fillna('[]') 

In [6]:
# Nan값 제거된 것 확인
metadata.article_tag_list.isnull().sum() 

0

In [7]:
metadata.head()

,magazine_id,user_id,title,article_tag_list,display_url,sub_title,reg_ts,article_id,id,date,hour,weekday,view,recent_view
0,8982.0,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","['여행', '호주', '국립공원']",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1474944427000.0,782,@bookdb_782,2016-09-27,11:47:07,Tue,0.0,0.0
1,12081.0,@kohwang56,[시] 서러운 봄,"['목련꽃', '아지랑이', '동행']",https://brunch.co.kr/@kohwang56/81,NaN,1463092749000.0,81,@kohwang56_81,2016-05-13,07:39:09,Fri,1.0,1.0
2,0.0,@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1447997287000.0,4,@hannahajink_4,2015-11-20,14:28:07,Fri,0.0,0.0
3,16315.0,@bryceandjuli,싫다,"['감정', '마음', '위로']",https://brunch.co.kr/@bryceandjuli/88,NaN,1491055161000.0,88,@bryceandjuli_88,2017-04-01,22:59:21,Sat,2.0,0.0
4,29363.0,@mijeongpark,Dubliner#7,"['유럽여행', '더블린', '아일랜드']",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1523292942000.0,34,@mijeongpark_34,2018-04-10,01:55:42,Tue,1.0,0.0


## metadata preprocessing 

In [8]:
# """
# String화된 list -> String의 리스트로 변환
# """
# for i in range(len(metadata)):
#     metadata['article_tag_list'][i] = metadata['article_tag_list'][i].replace('[', '')
#     metadata['article_tag_list'][i] = metadata['article_tag_list'][i].replace(']', '')
#     metadata['article_tag_list'][i] = metadata['article_tag_list'][i].replace("'", '')
    
#     metadata['article_tag_list'][i] = metadata['article_tag_list'][i].split(',')

In [422]:
re.findall(r"[\w]+", metadata['article_tag_list'][0])[0][0]

'여'

In [423]:
"""
String화된 list -> String의 리스트로 변환
"""
for i in range(len(metadata)):
    metadata['article_tag_list'][i] = re.findall(r"[\w]+", metadata['article_tag_list'][i])
    #for j in range(len(metadata['article_tag_list'][i])):
        #metadata['article_tag_list'][i][j] = metadata['article_tag_list'][i][j].replace("'", "")
        

In [424]:
metadata_itemtags = metadata['article_tag_list'].tolist()
metadata_itemtags #이중리스트형태

[['여행', '호주', '국립공원'],
 ['목련꽃', '아지랑이', '동행'],
 [],
 ['감정', '마음', '위로'],
 ['유럽여행', '더블린', '아일랜드'],
 ['석유에너지', '베네수엘라', '경제'],
 ['입찰', '유치권', '부동산경매변호사'],
 ['사랑', '연애', 'rain'],
 ['메일', '출판사'],
 ['도시애벌레', '공부', '동시'],
 ['패션', '에세이', '경제'],
 ['불꽃축제', '불꽃', '여의도'],
 ['주머니', '동시', '빵가게'],
 ['생각', '과거', '무시'],
 ['단상', '여행', '생각'],
 ['시험', '아침조회시간'],
 ['감성에세이', '사랑', '첫사랑'],
 ['애견놀이터', '애견', '애견카페'],
 [],
 ['엄마', '취향', '효도'],
 ['칼국수', '대화', '저항'],
 ['세계여행', '여행', '행복'],
 ['버스', '지하철', '친구'],
 ['그림자', '하이에나', '도시'],
 ['자작소설', '창작'],
 ['팀워크', '성과', '개인기'],
 ['브랜드', '브랜딩', '마케팅'],
 ['일자리', '청년실업', '저출산'],
 ['취업', '사회생활', '결혼'],
 [],
 ['장애엄마', '부모교육', '새학기'],
 ['젠트리피케이션', '주거문화', '서울'],
 ['크라쿠프', '소금광산', '유럽여행'],
 ['출판', '사회복지사', '교육'],
 ['승리', '아이돌', '추락'],
 ['문화극장', '극장'],
 ['월요일', '스트레스', '회사'],
 ['드라마', '방송', '방송연예'],
 ['아프리카', '배낭여행', '세계여행'],
 ['세네갈요리', '요리', '세네갈'],
 ['샐러드', '음식', '놀이'],
 ['마늘', '여자', '엄마'],
 ['세상', '이해', '생각'],
 [],
 ['회사', '사람', '생각'],
 ['소설', '서평', '페미니즘'],
 ['캐릭터', '드

In [21]:
"""
태그중에 영어 등장 --> 우선 삭제. . .
"""
def isHangul(text):
    encText = text
    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
    return hanCount > 0
    

In [426]:
for i in range(len(metadata_itemtags)):
    for j in range(len(metadata_itemtags[i])):
        if isHangul(metadata_itemtags[i][j]) == False:
            print("delete", i, j, metadata_itemtags[i][j])
            del metadata_itemtags[i][j]

delete 7 2 rain
delete 99 2 study


In [427]:
# 영어 삭제되었음.
metadata_itemtags

[['여행', '호주', '국립공원'],
 ['목련꽃', '아지랑이', '동행'],
 [],
 ['감정', '마음', '위로'],
 ['유럽여행', '더블린', '아일랜드'],
 ['석유에너지', '베네수엘라', '경제'],
 ['입찰', '유치권', '부동산경매변호사'],
 ['사랑', '연애'],
 ['메일', '출판사'],
 ['도시애벌레', '공부', '동시'],
 ['패션', '에세이', '경제'],
 ['불꽃축제', '불꽃', '여의도'],
 ['주머니', '동시', '빵가게'],
 ['생각', '과거', '무시'],
 ['단상', '여행', '생각'],
 ['시험', '아침조회시간'],
 ['감성에세이', '사랑', '첫사랑'],
 ['애견놀이터', '애견', '애견카페'],
 [],
 ['엄마', '취향', '효도'],
 ['칼국수', '대화', '저항'],
 ['세계여행', '여행', '행복'],
 ['버스', '지하철', '친구'],
 ['그림자', '하이에나', '도시'],
 ['자작소설', '창작'],
 ['팀워크', '성과', '개인기'],
 ['브랜드', '브랜딩', '마케팅'],
 ['일자리', '청년실업', '저출산'],
 ['취업', '사회생활', '결혼'],
 [],
 ['장애엄마', '부모교육', '새학기'],
 ['젠트리피케이션', '주거문화', '서울'],
 ['크라쿠프', '소금광산', '유럽여행'],
 ['출판', '사회복지사', '교육'],
 ['승리', '아이돌', '추락'],
 ['문화극장', '극장'],
 ['월요일', '스트레스', '회사'],
 ['드라마', '방송', '방송연예'],
 ['아프리카', '배낭여행', '세계여행'],
 ['세네갈요리', '요리', '세네갈'],
 ['샐러드', '음식', '놀이'],
 ['마늘', '여자', '엄마'],
 ['세상', '이해', '생각'],
 [],
 ['회사', '사람', '생각'],
 ['소설', '서평', '페미니즘'],
 ['캐릭터', '드로잉', '그림

## model load

In [428]:
ko_model = gensim.models.Word2Vec.load('./ko/ko.bin')

In [430]:
"""
wv.vocab에 없는 경우
--> 형태소 분석 / 삭제 (우선 삭제 후 진행)
"""
metadata_itemtags_after=[]
for i in range(len(metadata_itemtags)):
    metadata_itemtags_after.append([])
    for j in range(len(metadata_itemtags[i])):
        if metadata_itemtags[i][j] not in ko_model.wv.vocab:
            continue
        metadata_itemtags_after[i].append(metadata_itemtags[i][j])

In [431]:
# 목련꽃, 부동산경매변호사 등 삭제되었음
metadata_itemtags_after

[['여행', '호주', '국립공원'],
 ['동행'],
 [],
 ['감정', '마음', '위로'],
 ['더블린', '아일랜드'],
 ['경제'],
 ['입찰', '유치권'],
 ['사랑', '연애'],
 ['메일', '출판사'],
 ['공부', '동시'],
 ['패션', '에세이', '경제'],
 ['불꽃', '여의도'],
 ['주머니', '동시'],
 ['생각', '과거', '무시'],
 ['단상', '여행', '생각'],
 ['시험'],
 ['사랑', '첫사랑'],
 ['애견'],
 [],
 ['엄마', '취향', '효도'],
 ['대화', '저항'],
 ['여행', '행복'],
 ['버스', '지하철', '친구'],
 ['그림자', '하이에나', '도시'],
 ['창작'],
 ['성과'],
 ['브랜드', '마케팅'],
 ['일자리'],
 ['취업', '사회생활', '결혼'],
 [],
 [],
 ['서울'],
 [],
 ['출판', '교육'],
 ['승리', '추락'],
 ['극장'],
 ['월요일', '스트레스', '회사'],
 ['드라마', '방송'],
 ['아프리카'],
 ['요리', '세네갈'],
 ['샐러드', '음식', '놀이'],
 ['마늘', '여자', '엄마'],
 ['세상', '이해', '생각'],
 [],
 ['회사', '사람', '생각'],
 ['소설', '서평', '페미니즘'],
 ['캐릭터', '드로잉', '그림'],
 [],
 ['일상', '단편'],
 ['비틀즈', '음악', '앨범'],
 ['싱가포르', '이민'],
 ['영화', '매트릭스', '리뷰'],
 ['인문학', '소프트웨어', '융합'],
 [],
 [],
 ['스트레스'],
 ['마약', '서평'],
 ['필리핀'],
 ['운동', '헬스', '건강'],
 ['상표'],
 ['자전거', '잠', '맥주'],
 ['사진', '편집'],
 [],
 ['착각', '성공'],
 ['시', '이별', '위로'],
 [],
 ['상표'],
 ['벌레', '단편'],

In [277]:
ko_model.wv.most_similar("여행")


[('항해', 0.6019408702850342),
 ('탐험', 0.5845087766647339),
 ('여행자', 0.5578233003616333),
 ('체류', 0.5551777482032776),
 ('방문', 0.5392764806747437),
 ('여정', 0.5236952900886536),
 ('순방', 0.5188455581665039),
 ('유람', 0.5148690938949585),
 ('돌아다니', 0.5122349262237549),
 ('하이킹', 0.48751986026763916)]

In [17]:
"""
word -> vector practice code
"""
#ko_model.wv.get_vector('여행')
#ko_model.wv.get_vector(tags[0][0])
ko_model.wv[tags[0][0]]

array([-0.22261088, -0.3179489 , -0.31348732,  0.30306804,  1.072162  ,
        0.71200967, -0.35552275,  2.8992279 ,  0.3236653 , -2.0428343 ,
        1.5717347 ,  0.39062178,  1.3248582 ,  1.0042733 , -1.1683393 ,
       -1.1125335 ,  1.1176921 ,  0.0915822 , -0.15841556,  1.1593411 ,
        0.04707694, -0.5005447 ,  0.814857  , -0.64887893,  0.4252706 ,
       -2.3010507 ,  0.29640567,  0.7389912 , -1.4511623 ,  0.1264299 ,
        0.6729722 , -2.0054646 ,  1.3086209 , -1.9933537 ,  2.0497043 ,
        0.03216873, -2.1521363 ,  1.1799654 , -0.63337827,  0.36581668,
        0.20355628, -0.15857908,  0.69844157, -1.1080979 , -1.4903637 ,
       -0.8328029 ,  0.7670144 , -2.2677085 , -1.519313  ,  0.4455183 ,
        2.7102494 , -0.08926437,  0.41304168, -0.04969173, -0.10132965,
       -3.5819385 ,  0.12846859, -1.333964  , -0.3622299 ,  1.9693674 ,
       -0.77796155, -0.67008424, -0.67353237,  1.0405833 , -0.3379165 ,
       -0.07150173, -0.5487609 , -0.1970717 ,  1.005932  , -0.11

In [18]:
# 각 단어가 200차원의 벡터형태. 그 단어 (보통) 3개가 리스트형태로 묶인 것이 태그. 
# 그 태그는 클러스터 한개로 생각할 수 있음
# 우선 클러스터 centroid끼리의 계산


In [432]:
"""
cluster_centroid = torch.zeros(4,200)
for i in range(len(tags)):# 0 ~ 9 태그 인덱스
    centroid = torch.zeros(200)
    for j in range(len(tags[i])): # 각 태그의 길이
        # 단어->벡터
        tmp = torch.tensor(ko_model.wv.get_vector(tags[i][j])) 
        centroid = (centroid + tmp) / len(tags[i])
        if j == len(tags[i]) - 1: #마지막 태그일 때
            print(tags[i])
            print(centroid)
            cluster_centroid[i] = centroid

"""
"""
각 글에서 tag하나하나를 vector, 그 vector들을 모아서 cluster
-->중심연결법 사용을 위해 cluster centroid 구함
"""
centroid = torch.zeros(len(metadata_itemtags_after), 200)

for i in range(len(metadata_itemtags_after)):
    if metadata_itemtags_after[i] == []:
        continue
    # 태그에 있는 단어들을 한번에 vector화
    tmp = torch.Tensor(ko_model.wv[metadata_itemtags_after[i]])
    centroid[i] = tmp.mean(dim=0)
    #print(tags_after[i])
print(centroid.shape)

torch.Size([100, 200])


In [433]:
"""
pairwise하게 유사도 계산
"""
#cos_sim = cosine_similarity(centroid,centroid)
man_dis = manhattan_distances(centroid,centroid)
#cos_sim.shape # centroid끼리 유사도 계산

In [434]:
man_dis

array([[  0.        , 149.20947688, 112.82483077, ..., 207.77365804,
        187.19069154, 130.37945352],
       [149.20947688,   0.        , 121.85711063, ..., 235.52980979,
        183.40702869, 145.05453866],
       [112.82483077, 121.85711063,   0.        , ..., 200.16047874,
        161.00097245,  82.438185  ],
       ...,
       [207.77365804, 235.52980979, 200.16047874, ...,   0.        ,
        231.4017977 , 185.24701834],
       [187.19069154, 183.40702869, 161.00097245, ..., 231.4017977 ,
          0.        , 187.88736517],
       [130.37945352, 145.05453866,  82.438185  , ..., 185.24701834,
        187.88736517,   0.        ]])

In [281]:
cos_sim = 1 - cos_sim
cos_sim

array([[ 0.0000000e+00,  8.4826541e-01,  1.0000000e+00,  1.0500790e+00,
         6.7981523e-01,  9.0501100e-01,  9.4869882e-01,  9.2221779e-01,
         8.6557925e-01,  9.0703732e-01],
       [ 8.4826541e-01, -1.1920929e-07,  1.0000000e+00,  8.4974837e-01,
         9.3642890e-01,  1.1591481e+00,  8.7549108e-01,  7.8046668e-01,
         9.7663021e-01,  6.5034544e-01],
       [ 1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00,
         1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00,
         1.0000000e+00,  1.0000000e+00],
       [ 1.0500790e+00,  8.4974837e-01,  1.0000000e+00, -1.1920929e-07,
         1.1038097e+00,  1.0330408e+00,  9.4942051e-01,  4.3633014e-01,
         9.9832988e-01,  7.7558118e-01],
       [ 6.7981523e-01,  9.3642890e-01,  1.0000000e+00,  1.1038097e+00,
         0.0000000e+00,  9.7752172e-01,  1.1386645e+00,  9.6804923e-01,
         9.5175415e-01,  9.3178588e-01],
       [ 9.0501100e-01,  1.1591481e+00,  1.0000000e+00,  1.0330408e+00,
   

In [435]:
# 타겟 글(0번)과 가장 유사한 글 = 4번째 인덱스
similar_index = (np.where(cos_sim[0] == sorted(cos_sim[0])[1]))[0][0]
similar_index

4

In [436]:
# 비슷한 글 3개 추천
similarity_rank_index = []
for rank in range(1,4):
    similarity_rank_index.append((np.where(cos_sim[0] == sorted(cos_sim[0])[rank]))[0][0])
print(similarity_rank_index)

[4, 1, 8]


In [438]:
metadata.iloc[similarity_rank_index]

,magazine_id,user_id,title,article_tag_list,display_url,sub_title,reg_ts,article_id,id,date,hour,weekday,view,recent_view
4,29363.0,@mijeongpark,Dubliner#7,"[유럽여행, 더블린, 아일랜드]",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1523292942000.0,34,@mijeongpark_34,2018-04-10,01:55:42,Tue,1.0,0.0
1,12081.0,@kohwang56,[시] 서러운 봄,"[목련꽃, 아지랑이, 동행]",https://brunch.co.kr/@kohwang56/81,NaN,1463092749000.0,81,@kohwang56_81,2016-05-13,07:39:09,Fri,1.0,1.0
8,28849.0,@elang8151,출간제의 받았습니다.,"[메일, 출판사]",https://brunch.co.kr/@elang8151/229,미안하지만 아직 학생입니다.,1539405177000.0,229,@elang8151_229,2018-10-13,13:32:57,Sat,128.0,0.0


# 궁극적으로 하고자하는 것 = dev.users/test.users의 타겟 유저가 읽었던 글 과 비슷한 글을 찾고자함.

In [8]:
#### dev.users --> target_df

In [41]:
target_dev = pd.read_csv('./res/target_dev.csv')
target_dev.head()

,keyword_list,following_list,user_id,following_cnt,following_cnt_rank,read,recent,recent_following,recent_magazine,recent_tag,recent_interest,recent_f_ratio,recent_m_ratio,recent_p_ratio,recent_r_ratio
0,[],"['@potatohands', '@ggpodori', '@chae-pulib', '...",#a6f7a5ff90a19ec4d583f0db1836844d,53.0,523.0,"['@bookguru_31', '@backkoo_101', '@theedit_546...","['@daljasee_222', '@shindong_38', '@daljasee_2...",{'': 177},"Counter({8274.0: 9, 38306.0: 4, 40519.0: 3, 33...","Counter({'[]': 4, ""['연애', '감정', '사랑']"": 1, ""['...","['[]', ""['연애', '감정', '사랑']"", ""['기획', '회사', '전략...",1.0,0.56,0.96,0.23
1,[],"['@seochogirl', '@cloudocloud', '@youngmusic',...",#0303a4d2ad9570cfb4d1658b684ad08b,140.0,436.0,"['@minsungdkim_222', '@minsungdkim_223', '@min...","['@youngmusic_54', '@soandso_77', '@seochogirl...",{'': 51},"Counter({12804.0: 3, 24781.0: 2, 36426.0: 2, 4...","Counter({""['연애', '사랑', '이별']"": 1, ""['입학', '준비'...","[""['연애', '사랑', '이별']"", ""['입학', '준비', '육아']"", ""...",1.0,0.47,0.82,0.76
2,[],"['@merryseo', '@workerhanee', '@daltokki', '@h...",#6516c871163aeafd8a51d5db44148aca,4.0,572.0,"['@workerhanee_258', '@taekangk_62', '@taekang...","['@workerhanee_258', '@taekangk_62', '@workerh...",{'': 13},"Counter({26129.0: 2, 37613.0: 2, 34075.0: 1, 3...","Counter({""['퇴사', '일러스트', '에세이']"": 1, ""['공감에세이'...","[""['퇴사', '일러스트', '에세이']"", ""['공감에세이', '육아에세이', ...",1.0,0.54,0.77,0.46
3,[],"['@withssum', '@yooatic84', '@hoteller', '@let...",#c4161080cbad8739f8f5ae92e8839250,6.0,570.0,"['@psychiatricnews_124', '@psychiatricnews_125...","['@tamarorim_101', '@tamarorim_100', '@tamaror...",{'': 142},"Counter({34975.0: 10, 40258.0: 5, 39067.0: 5, ...","Counter({'[]': 5, ""['심리', '치유', '에세이']"": 2, ""[...","['[]', ""['심리', '치유', '에세이']"", ""['디자인', '표지']"",...",1.0,0.29,0.94,0.11
4,"[{'cnt': 1, 'keyword': '6년차'}, {'cnt': 2, 'key...","['@jjollae', '@seanseda', '@munchi', '@yoonsol...",#b757e6adac7edfb445ac8c776b1d02bc,39.0,537.0,"['@makebotai_4', '@kakao-it_187', '@bellrings_...","['@tamarorim_135', '@tamarorim_136', '@zagni_2...",{'': 3},"Counter({4905.0: 2, 24941.0: 1})","Counter({""['신혼부부', '신혼여행']"": 1, '[]': 1, ""['좋은...","[""['신혼부부', '신혼여행']"", '[]', ""['좋은글', '팟캐스트', '정...",1.0,1.00,1.00,0.00


### 타겟유저가 읽은 글들의 태그 count --> top-5개 추림

In [42]:
target_read_tag = []

for idx in target_dev['user_id'].values.tolist():
    # read_list- > target user의 read_list

    read_list = re.findall(r"[\w@-]+", target_dev[target_dev['user_id']==idx]['recent'].values[0][:])
            
    keyword_list = metadata[metadata['id'].isin(read_list)]['article_tag_list'].tolist()
    for i in range(len(keyword_list)):
        keyword_list[i] = re.findall(r"[\w]+", keyword_list[i])
    read_tag = sum(keyword_list, [])
    if idx == '#f109244f9a9fb8947472736f89df59e3':
        print(read_tag)     
    # 각 target user가 일정 기간 동안 읽은 글의 태그들의 빈도수 저장
    frequency = Counter(read_tag)
    # 타겟 유저가 읽은 글의 태그 모음.
    target_read_tag.append(frequency)

['영화', '영화리뷰', '다큐멘터리', '고양이', '길고양이', '고양이사진', '직장인', '독서모임', '일', '진로', '적성', '적성검사', '맥주', 'AI', '건강', '행복', '스트레스', '정리', '비용', '패션에세이', '연재만화', '만화', '이웃집유부녀', '만화', '이웃집유부녀', '상식', '커피', '홍차', '다방', '투자', '벤처투자금', '스타트업', '이문세', '가사', '잔나비김윤희', '아이디어', '스타트업', '리뷰', '스타벅스', '카페', '커피빈', '안희정', '이슈', '출판', '출판사', '신간', '오디오클립', '콘텐츠', '네이버', '부산', '부산여행', '해운대', '고양이', '고양이사진', '길고양이', '독서모임', '공부', '직장인', '사진', 'iPhone', '아이폰', '영화', '영화리뷰', '영화이야기', '독서', '자기계발', '글쓰기', '무선충전기', '이어폰', '어둠', '밤', '칼럼']


In [43]:
#sorted(target_read_tag[96].items(), key=lambda pair: pair[1], reverse=True)
## 타겟 유저의 interest tag top-5
target_top5_tmp= []
for i in range(len(target_read_tag)):
    # 이중 리스트형태로 저장.
    target_top5_tmp.append(sorted(target_read_tag[i], key=target_read_tag[i].get, reverse=True)[0:5]
)

In [ ]:
target_top5_tmp[8:20]

In [44]:
"""
태그중에 영어 등장 --> 우선 삭제. . .
"""
def isHangul(text):
    encText = text
    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
    return hanCount > 0
    

In [45]:
"""
영어
--> 삭제
"""
target_top5_tmp2 = []
for i in range(len(target_top5_tmp)):
    target_top5_tmp2.append([])
    for j in range(len(target_top5_tmp[i])):
        if isHangul(target_top5_tmp[i][j]) == False:
            print("delete", i, j, target_top5_tmp[i][j])
            continue
        target_top5_tmp2[i].append(target_top5_tmp[i][j])

delete 8 0 IT
delete 8 2 java
delete 8 4 AI
delete 37 4 UX
delete 40 2 UX
delete 75 0 UX
delete 75 2 UI
delete 97 0 IT
delete 105 3 UX
delete 106 1 IT
delete 107 3 UX
delete 181 0 UX
delete 193 0 HR
delete 196 1 IT
delete 208 1 UI
delete 218 3 UX
delete 231 2 UX
delete 249 1 IT
delete 250 4 KBS
delete 283 2 UI
delete 283 3 UX
delete 292 0 UX
delete 292 1 UI
delete 298 1 UX
delete 298 4 UI
delete 318 2 UI
delete 318 3 UX
delete 320 2 AI
delete 323 1 AI
delete 323 3 PM
delete 323 4 IT
delete 325 2 UX
delete 334 2 IT
delete 385 1 UI
delete 387 4 Paris
delete 398 3 UX
delete 415 2 IT
delete 415 4 UI
delete 424 2 UX
delete 427 2 IT
delete 475 3 IT
delete 482 2 UX
delete 482 3 UI
delete 486 3 AI
delete 494 3 UX
delete 498 4 UX
delete 516 1 UX
delete 544 0 UX
delete 552 1 IT
delete 594 3 IT
delete 609 2 IT
delete 646 4 TIP
delete 665 3 UX
delete 687 3 UI
delete 695 1 HCI
delete 701 0 MBTI
delete 707 0 UX
delete 709 1 UX
delete 709 2 UI
delete 709 3 IT
delete 733 3 UX
delete 734 1 UX
delete 73

In [46]:
ko_model = gensim.models.Word2Vec.load('./ko/ko.bin')

In [47]:
"""
wv.vocab에 없는 경우
--> 형태소 분석 / 삭제 (우선 삭제 후 진행)
"""
target_top5_tag=[]
for i in range(len(target_top5_tmp2)):
    target_top5_tag.append([])
    for j in range(len(target_top5_tmp2[i])):
        if target_top5_tmp2[i][j] not in ko_model.wv.vocab:
            continue
        target_top5_tag[i].append(target_top5_tmp2[i][j])

In [ ]:
## 전처리 완료한 타겟 유저의 interest tag
# target_top5_tag

## word2vec 벡터화

In [48]:
"""
관심 tag하나하나를 vector, 그 vector들을 모아서 cluster

"""

for i in range(len(target_top5_tag)):
    if target_top5_tag[i] == []:
        continue
    # 태그에 있는 단어들을 한번에 vector화
    # target_top5_tag: 3차원 tensor형태(3000, 각자 태그 개수, 2000차원벡터)
    target_top5_tag[i] = torch.Tensor(ko_model.wv[target_top5_tag[i]])
    

## metadata item에 대한 워드임배딩

In [49]:
meta_tag_list = metadata['article_tag_list'].tolist()
for i in range(len(meta_tag_list)):
    meta_tag_list[i] = re.findall(r"[\w]+", meta_tag_list[i])

In [50]:
"""
영어
--> 삭제
"""
meta_tag_list_tmp = []
for i in range(len(meta_tag_list)):
    meta_tag_list_tmp.append([])
    for j in range(len(meta_tag_list[i])):
        if isHangul(meta_tag_list[i][j]) == False:
            print("delete", i, j, meta_tag_list[i][j])
            continue
        meta_tag_list_tmp[i].append(meta_tag_list[i][j])


elete 616972 2 IT
delete 616976 1 tea
delete 616995 2 POP
delete 617032 1 Santiago
delete 617048 0 CEO
delete 617060 1 piano
delete 617090 0 MBA
delete 617098 0 EPL
delete 617116 1 EPL
delete 617142 0 CEO
delete 617198 2 OST
delete 617264 0 IT
delete 617281 0 O2O
delete 617294 0 IT
delete 617331 2 life
delete 617335 2 Google
delete 617336 0 IT
delete 617357 0 UI
delete 617363 2 IT
delete 617423 2 coffee
delete 617433 0 IT
delete 617433 1 Christmas
delete 617475 2 IT
delete 617548 0 UI
delete 617569 1 IT
delete 617640 2 IT
delete 617642 1 digital
delete 617655 0 LG
delete 617677 2 IT
delete 617703 0 poem
delete 617722 0 art
delete 617881 1 think
delete 617881 2 TED
delete 617919 2 TV
delete 617935 1 photograph
delete 617971 2 TV
delete 617980 0 PD
delete 618101 2 POD
delete 618172 2 LG
delete 618180 1 SNS
delete 618182 0 UX
delete 618185 2 MSG
delete 618216 2 IT
delete 618221 1 TED
delete 618222 1 design
delete 618283 2 TIP
delete 618291 1 man
delete 618291 2 you
delete 618302 0 SNS
del

In [51]:
"""
wv.vocab에 없는 경우
--> 형태소 분석 / 삭제 (우선 삭제 후 진행)
"""
metadata_tag = []
for i in range(len(meta_tag_list_tmp)):
    metadata_tag.append([])
    for j in range(len(meta_tag_list_tmp[i])):
        if meta_tag_list_tmp[i][j] not in ko_model.wv.vocab:
            continue
        metadata_tag[i].append(meta_tag_list_tmp[i][j])

#### metadata_tag 워드 임배딩

In [52]:
for i in range(len(metadata_tag)):
    if metadata_tag[i] == []:
        continue
    # 태그에 있는 단어들을 한번에 vector화
    # target_top5_tag: 3차원 tensor형태(3000, 각자 태그 개수, 2000차원벡터)
    metadata_tag[i] = torch.Tensor(ko_model.wv[metadata_tag[i]])
    

1. 타겟 유저 0번의 관심 태그와 글 태그 클러스터간 최단연결 거리 = 해당 유저가 글에 가질 관심도로 간주
2. 타겟 유저 0번에 대한 글 643111개 모두 수행.target_read_tag
3. 643111에 대한 최단연결 거리들 중 제일 가까운 100개 추천.

In [ ]:
# 타겟 글(0번)과 가장 유사한 글 = 4번째 인덱스
#similar_index = (np.where(cos_sim[0] == sorted(cos_sim[0])[1]))[0][0]
#similar_index

In [59]:
dis = torch.zeros(len(metadata_tag))
for target in range(1): 
    for item in range(5):
        dis[item] = np.amin(1 - cosine_similarity(target_top5_tag[target], metadata_tag[item]))

TypeError: can't assign a numpy.float32 to a torch.FloatTensor

In [ ]:
# 비슷한 글 3개 추천
similarity_rank_index = []
for rank in range(1,100):
    similarity_rank_index.append((np.where(cos_sim[0] == sorted(cos_sim[0])[rank]))[0][0])
print(similarity_rank_index)

In [36]:
for target in range(len(target_top5_tag)):
    for item in range(len(metadata_tag)):
        cosine_similarity(target_top5_tag[target], metadata_tag[item])

array([[ 0.24306536,  0.05163486, -0.2586939 ],
       [ 0.25922763,  0.16066436, -0.07639411],
       [ 0.2969677 , -0.06987188, -0.069573  ],
       [ 0.19989194,  0.08982993, -0.11904066]], dtype=float32)

In [57]:
""" 
target_df 생성 (users에 없는 user_id 추가) 
: 매개변수에는 dev_users_list / test_users_list 와 users. 
""" 
def target_df_generator(target_users_list, users):
    print('1. target user의 dataframe 생성 중')
    target_df = users[users['user_id'].isin(target_users_list)]
    
    for target_user in target_users_list:
        if (target_user in target_df['user_id'].tolist()) == False:
            new_df = pd.DataFrame({'following_list':[[]], 'user_id':[target_user], 'keyword_list':[[]]})
            target_df = target_df.append(new_df)
    
    print('target user의 dataframe 생성 완료')
    return target_df


In [62]:
"""
전체 기간동안 target user가 읽은 article 저장
"""
def target_read_article(target_df, read):
    print('2. 전체 기간동안 target user가 읽은 article 저장 중\n')

    target_read_article = []
    for idx in target_df['user_id'].values.tolist(): # user_id한개씩 루프
        read_list = read[read['user_id']==idx]
        read_list = read_list[read_list['article_id'].str.contains('@')==True]['article_id'].tolist()#해당 user가 읽은 글 목록
        target_read_article.append(read_list)

    target_df['read'] = target_read_article # 읽은 글 
            
    print('전체 기간동안 target user가 읽은 article 저장 완료\n')
    return target_df


In [64]:
users = pd.read_csv('./res/users.csv')
read =  pd.read_csv('./res/read.csv')
metadata = pd.read_csv('./res/metadata.csv')

In [179]:
target_dev = pd.read_csv('./res/target_dev.csv')
for i in range(len(target_dev)):
    target_dev['read'][i] = re.findall(r"[\w@-]+", target_dev['read'][i])
# [xy]: 문자 선택을 표현하며 x 와 y 중에 하나
# \w: word 를 표현하며 알파벳 + 숫자 + _ 중의 한 문자
# [\w']+: (알파벳,숫자,_) + ' + ~ 로 시작하는 것 찾음

In [181]:
import re
# 우선 recent interest tag를 한 벡터에 다 넣어버림......좀 이상하지만....
for i in range(len(target_dev)):
    target_dev['recent_interest'][i] = re.compile('[가-힣]+').findall(target_dev['recent_interest'][i]) 

['연애', '취업', '자소서', '사랑', '에세이']

In [184]:
#10개만 먼저
target_dev = target_dev[0:10]

In [188]:
target_tag = target_dev['recent_interest'].tolist()
#target_tag

In [189]:
"""
태그중에 영어 등장 --> 우선 삭제. . .
"""
def isHangul(text):
    encText = text
    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
    return hanCount > 0

for i in range(len(target_tag)):
    for j in range(len(target_tag[i])):
        if isHangul(target_tag[i][j]) == False:
            del target_tag[i][j]
    

In [191]:
ko_model = gensim.models.Word2Vec.load('./ko/ko.bin')

In [197]:
"""
wv.vocab에 없는 경우
--> 형태소 분석 / 삭제 (우선 삭제 후 진행)
"""
tags_after=[]
for i in range(len(target_tag)):
    tags_after.append([])
    for j in range(len(target_tag[i])):
        if target_tag[i][j] not in ko_model.wv.vocab:
            continue
        tags_after[i].append(target_tag[i][j])
        
# 관심 태그 전처리 완료!
tags_after

[['연애', '감정', '사랑', '기획', '회사', '전략', '에세이', '취업', '관계'],
 ['연애', '사랑', '이별', '입학', '준비', '육아', '다람쥐', '선생님', '공작', '드로잉'],
 ['퇴사', '에세이', '에세이', '에세이', '에세이', '회사', '후임'],
 ['심리', '치유', '에세이', '디자인', '표지', '구분', '일요일', '월요일', '내일'],
 ['정리'],
 ['연애', '결혼', '데이트', '결혼식', '사회자', '진심', '연애', '노력', '사랑', '연애', '고백'],
 ['멜론', '키즈', '디자인', '멜론', '페스티벌', '직장', '이직', '회사'],
 ['남자', '쇼핑몰', '패션', '패션', '스타일', '코디', '사람', '직장인', '직장인', '백수'],
 ['자바', '투자', '정부', '애플', '자동차', '인공지능'],
 ['마케팅', '글쓰기', '직장인', '백수', '배달', '직장인']]

In [198]:
"""
관심 tag하나하나를 vector, 그 vector들을 모아서 cluster
-->중심연결법 사용을 위해 cluster centroid 구함
"""
\
for i in range(len(tags_after)):
    if tags_after[i] == []:
        continue
    # 태그에 있는 단어들을 한번에 vector화
    tmp = torch.Tensor(ko_model.wv[tags_after[i]])
    centroid[i] = tmp.mean(dim=0)
    #print(tags_after[i])
print(centroid.shape)

torch.Size([10, 200])


In [200]:
# 최단연결법으로 바꾸려고함.

# ㅂ베각 클러스터에 n개의 벡터들이 존재하는데,
# ㅈ제일 가까운 벡터간 거리를 구하고싶으니까.
# ㅇ우선 모든 벡터정보를 갖고있어야함.
# ㄱ원래 tags_after와 같은 크기의 이중리스트에 벡터들을 저장--> 삼중리스트가 되겠군.
#
[# tags_after = '연애', '감정', '사랑', '기획', '회사', '전략', '에세이', '취업', '관계'],
#  비교하고자하는 글의 태그item_tags =  ['감정', '마음', '위로']

# cos_dis = 1 - cos_sim(tags_after, item_tags) 하나하나 다 계산해서 저장
# ㅇcos_dis을 sort한 후, top-k(원래는100인데, 여기서는 우선 3만 해보자)개의 글 아이디만 뽑아내면됨.

# gtags)_after 행마다 진행해야할듯 ㅠ 관심 태그 개수가 모두 달라서,,,

tensor([[ 0.0348, -0.6137,  0.2684,  ...,  0.0067, -0.1362, -0.2836],
        [-0.3402, -0.5622,  0.3837,  ...,  0.0986, -0.0289, -0.1762],
        [-0.3292, -0.8880,  0.2804,  ..., -0.0411, -0.0162, -0.5970],
        ...,
        [-0.3842, -0.7099,  0.7794,  ...,  0.4746,  0.5367, -0.6770],
        [ 0.1950, -0.2473, -0.5680,  ...,  0.1981, -0.4867,  0.0361],
        [-0.0684, -0.1779,  0.1070,  ...,  0.1344,  0.5994, -0.1948]])

In [204]:

tags_vector = torch.Tensor(ko_model.wv[tags_after[0]])


In [220]:
# item의 태그
tags_after


[['여행', '호주', '국립공원'],
 ['동행'],
 [],
 ['감정', '마음', '위로'],
 ['더블린', '아일랜드'],
 ['경제'],
 ['입찰', '유치권'],
 ['사랑', '연애'],
 ['메일', '출판사'],
 ['공부', '동시'],
 ['패션', '에세이', '경제'],
 ['불꽃', '여의도'],
 ['주머니', '동시'],
 ['생각', '과거', '무시'],
 ['단상', '여행', '생각'],
 ['시험'],
 ['사랑', '첫사랑'],
 ['애견'],
 [],
 ['엄마', '취향', '효도'],
 ['대화', '저항'],
 ['여행', '행복'],
 ['버스', '지하철', '친구'],
 ['그림자', '하이에나', '도시'],
 ['창작'],
 ['성과'],
 ['브랜드', '마케팅'],
 ['일자리'],
 ['취업', '사회생활', '결혼'],
 [],
 [],
 ['서울'],
 [],
 ['출판', '교육'],
 ['승리', '추락'],
 ['극장'],
 ['월요일', '스트레스', '회사'],
 ['드라마', '방송'],
 ['아프리카'],
 ['요리', '세네갈'],
 ['샐러드', '음식', '놀이'],
 ['마늘', '여자', '엄마'],
 ['세상', '이해', '생각'],
 [],
 ['회사', '사람', '생각'],
 ['소설', '서평', '페미니즘'],
 ['캐릭터', '드로잉', '그림'],
 [],
 ['일상', '단편'],
 ['비틀즈', '음악', '앨범'],
 ['싱가포르', '이민'],
 ['영화', '매트릭스', '리뷰'],
 ['인문학', '소프트웨어', '융합'],
 [],
 [],
 ['스트레스'],
 ['마약', '서평'],
 ['필리핀'],
 ['운동', '헬스', '건강'],
 ['상표'],
 ['자전거', '잠', '맥주'],
 ['사진', '편집'],
 [],
 ['착각', '성공'],
 ['시', '이별', '위로'],
 [],
 ['상표'],
 ['벌레', '단편'],

In [222]:

tag_item = torch.zeros(len(tags_after), 200)

for i in range(len(tags_after)):
    if tags_after[i] == []:
        continue
    # 태그에 있는 단어들을 한번에 vector화
    tag_item[i] = torch.Tensor(ko_model.wv[tags_after[i]])


RuntimeError: expand(torch.FloatTensor{[3, 200]}, size=[200]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

In [224]:
tag_item0 = torch.Tensor(ko_model.wv[tags_after[0]])

In [234]:
"""
pairwise하게 유사도 계산해서 제일 가까운걸로 선택.
"""
pairwise_cos = 1 - cosine_similarity(tags_vector,tag_item0)

In [240]:
cosine_similarity(tags_vector,tag_item0)

array([[ 0.24306536,  0.05163486, -0.2586939 ],
       [ 0.09060258, -0.14001034, -0.07869197],
       [ 0.2969677 , -0.06987188, -0.069573  ],
       [ 0.09351079,  0.01961989, -0.01891658],
       [ 0.12378131,  0.18405223,  0.05976969],
       [ 0.02614605,  0.04482384,  0.04429719],
       [ 0.19989194,  0.08982993, -0.11904066],
       [ 0.25922763,  0.16066436, -0.07639411],
       [-0.00725169,  0.13895418, -0.1298497 ]], dtype=float32)

In [237]:
np.amin(pairwise_cos) # 이게 그럼 0번 유저의 인터레스트와 0번 아이템의 거리. 이걸 n번 아이템까지 반복하고 제일 가까운 k개 아이템을 추천해주면 될듯.

0.7030323

In [236]:
np.where(pairwise_cos == np.amin(pairwise_cos)) 

(array([2]), array([0]))